In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.mllib.fpm import FPGrowth
import re
from random import random
from pyspark import SparkContext

In [2]:
# config = pyspark.SparkConf().set('spark.executor.memory', '512M').set('spark.cores.max', '2')\
#             .set('spark.port.maxRetries','200') 
import os
port = 4040 + hash(os.getcwd().split("/")[2])%200
config = pyspark.SparkConf().set('spark.executor.memory', '512M')\
         .set('spark.cores.max', '2')\
         .set('spark.port.maxRetries','200')\
         .set('spark.ui.port', port)    
print port,     os.getcwd().split("/")[2]
sc = pyspark.SparkContext(appName="", master='',conf = config )

4150 ajp218


In [3]:
data = sc.textFile("dataP/visit_data.txt")

In [4]:
print data.take(10)

[u'1000,1011,Fruit', u'1000,10136,Carbonated Beverages', u'1000,10136,Dairy', u'1000,10136,Snack Foods', u'1000,10173,Bread', u'1000,10173,Candy', u'1000,10173,Canned Sardines', u'1000,10173,Electrical', u'1000,10173,Frozen Entrees', u'1000,10173,Meat']


In [5]:
data = sc.parallelize([ ["k1", "a"],["k1", "b"],["k2", "a"],["k2", "c"] ])
groupedData = data.groupByKey()
for e in groupedData.collect():
    print "------------\n", e[0]
    for j in e[1]:
        print j
        

------------
k1
a
b
------------
k2
a
c


In [6]:
data = sc.textFile("/home/globalscratch/ISE495/lab_04_FP/visit_data.txt")
print data.take(10),"\n\n"
def splitData(line):
    d = line.split(",")
    key = d[0]+"_"+d[1]
    value = d[2]
    return (key,value)

newdata = data.map(splitData)
print newdata.take(10), "\n\n"

print newdata.groupByKey().take(4)
for e in newdata.groupByKey().take(4):
    print e[0],":", [v for v in e[1]]

    
def keepTheValue(kv):
    out = []
    for v in kv[1]:
        out.append(v)
    return out
print "\n\n",newdata.groupByKey().map(keepTheValue).take(10)   

print "\n\n",newdata.groupByKey().map(lambda kv: [v for v in kv[1]] ).take(10)   


transactions = newdata.groupByKey().map(keepTheValue)

[u'1000,1011,Fruit', u'1000,10136,Carbonated Beverages', u'1000,10136,Dairy', u'1000,10136,Snack Foods', u'1000,10173,Bread', u'1000,10173,Candy', u'1000,10173,Canned Sardines', u'1000,10173,Electrical', u'1000,10173,Frozen Entrees', u'1000,10173,Meat'] 


[(u'1000_1011', u'Fruit'), (u'1000_10136', u'Carbonated Beverages'), (u'1000_10136', u'Dairy'), (u'1000_10136', u'Snack Foods'), (u'1000_10173', u'Bread'), (u'1000_10173', u'Candy'), (u'1000_10173', u'Canned Sardines'), (u'1000_10173', u'Electrical'), (u'1000_10173', u'Frozen Entrees'), (u'1000_10173', u'Meat')] 


[(u'423_5230', <pyspark.resultiterable.ResultIterable object at 0x7fe4e8526c10>), (u'736_7121', <pyspark.resultiterable.ResultIterable object at 0x7fe51dc5d310>), (u'899_1930', <pyspark.resultiterable.ResultIterable object at 0x7fe51dc5d390>), (u'899_1936', <pyspark.resultiterable.ResultIterable object at 0x7fe51dc5d410>)]
423_5230 : [u'Fruit', u'Snack Foods', u'Vegetables']
736_7121 : [u'Canned Soup', u'Dairy', u'Eggs', u

In [7]:
model = FPGrowth.train(transactions, minSupport=0.05, numPartitions=10)

result = model.freqItemsets().collect()
print('---------------------------')
for fi in result:
    print(fi)

---------------------------
FreqItemset(items=[u'Hot Beverages'], freq=3830)
FreqItemset(items=[u'Electrical'], freq=6158)
FreqItemset(items=[u'Vegetables'], freq=23198)
FreqItemset(items=[u'Candy'], freq=6157)
FreqItemset(items=[u'Candy', u'Vegetables'], freq=2746)
FreqItemset(items=[u'Snack Foods'], freq=22229)
FreqItemset(items=[u'Snack Foods', u'Vegetables'], freq=9690)
FreqItemset(items=[u'Eggs'], freq=3697)
FreqItemset(items=[u'Dairy'], freq=13819)
FreqItemset(items=[u'Dairy', u'Snack Foods'], freq=5638)
FreqItemset(items=[u'Dairy', u'Vegetables'], freq=5874)
FreqItemset(items=[u'Pain Relievers'], freq=3156)
FreqItemset(items=[u'Bathroom Products'], freq=6074)
FreqItemset(items=[u'Hygiene'], freq=3155)
FreqItemset(items=[u'Beer and Wine'], freq=6066)
FreqItemset(items=[u'Meat'], freq=11543)
FreqItemset(items=[u'Meat', u'Dairy'], freq=3146)
FreqItemset(items=[u'Meat', u'Snack Foods'], freq=4726)
FreqItemset(items=[u'Meat', u'Vegetables'], freq=5100)
FreqItemset(items=[u'Paper Prod

In [8]:
sc.stop()